<a href="https://colab.research.google.com/github/PriiiAiVerse/PriiiAiVerse/blob/main/Mistral_based_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required libraries
!pip install -q transformers langchain duckduckgo-search wikipedia pydantic accelerate bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00


In [2]:
# ✅ Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain.prompts import PromptTemplate
from duckduckgo_search import DDGS
import wikipedia
from datetime import datetime
from typing import Dict
import torch

In [3]:
# ✅ Load Mistral model in 4-bit quantized mode
model_id = "mistralai/Mistral-7B-Instruct-v0.2"


#!pip install -U -q bitsandbytes


In [4]:


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    repetition_penalty=1.1
)



tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
# ✅ LangChain-compatible wrapper
def hf_llm(input_text: str) -> str:
    prompt_text = input_text if isinstance(input_text, str) else input_text.get("query", "")
    return pipe(prompt_text)[0]['generated_text']

# ✅ Prompt Template
template = """### Instruction:
You are a research assistant. Please provide a concise research summary for the following topic: "{query}"

### Format:
Topic: ...
Summary: ...
Suggested tools: [Wikipedia, DuckDuckGo, Save-to-file]

### Response:"""

prompt = PromptTemplate.from_template(template)


In [7]:

# ✅ Tools
def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=3)
    except Exception:
        return "No Wikipedia summary found."

def ddg_search(query: str) -> str:
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results.append(f"- {r['title']}: {r['body']}")
    return "\n".join(results) if results else "No DuckDuckGo results."

def save_to_txt(data: str, filename: str = "research_output.txt"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_text = f"--- Research Output ---\nTimestamp: {timestamp}\n\n{data}\n\n"
    with open(filename, "a", encoding="utf-8") as f:
        f.write(formatted_text)
    return f"✅ Data saved to {filename}"



In [10]:
# ✅ Tool runner logic
def tool_runner(response: str, query: str) -> Dict[str, str]:
    results = {"model_response": response}
    if "Wikipedia" in response:
        results["wikipedia"] = wiki_search(query)
    if "DuckDuckGo" in response:
        results["duckduckgo"] = ddg_search(query)
    if "Save-to-file" in response:
        combined = f"{response}\n\nWikipedia Info:\n{results.get('wikipedia', '')}\n\nDuckDuckGo Info:\n{results.get('duckduckgo', '')}"
        results["save_result"] = save_to_txt(combined)
    return results

# ✅ Pipeline
user_query = input("🔍 Enter research topic: ")

chain = RunnableSequence(
    prompt,
    RunnableLambda(lambda x: x.to_string()),  # Fix: extract prompt as string
    RunnableLambda(hf_llm),
    RunnableLambda(lambda res: tool_runner(res, user_query)),
)

# ✅ Run the chain
results = chain.invoke({"query": user_query})

# ✅ Display results
print("\n📌 Model Response:\n", results['model_response'])
if 'wikipedia' in results:
    print("\n📚 Wikipedia:\n", results['wikipedia'])
if 'duckduckgo' in results:
    print("\n🌐 DuckDuckGo:\n", results['duckduckgo'])
if 'save_result' in results:
    print("\n💾", results['save_result'])


🔍 Enter research topic: how to understand genai


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📌 Model Response:
 ### Instruction:
You are a research assistant. Please provide a concise research summary for the following topic: "how to understand genai"

### Format:
Topic: ...
Summary: ...
Suggested tools: [Wikipedia, DuckDuckGo, Save-to-file]

### Response:

Topic: Understanding GenAI

Summary:
GenAI is a large language model developed by Mistral AI, based in Paris, France. It's part of the family of artificial intelligence models known as transformers and was trained on a vast dataset called "Mistral Dataset," which includes texts from various domains like books, websites, and multimedia content in multiple languages. To understand GenAI, it's essential to grasp its underlying technology—transformer models, its training data, its capabilities, and potential applications.

Understanding Transformer Models:
Transformer models, introduced by Vaswani et al. (2017), are a type of deep learning architecture designed specifically for handling long sequences of text. They employ atte